# IMPORTING THE LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error

# IMPORTING THE DATASET

In [ ]:
data = pd.read_csv('../input/stock-dataset/Stock.csv')

In [ ]:
data.head()

In [ ]:
data1 = data.reset_index()['close']

In [ ]:
data1

In [ ]:
data1.shape

In [ ]:
plt.plot(data1 , color = "purple")

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
data1 = scaler.fit_transform(np.array(data1).reshape(-1,1))

In [ ]:
data1

In [ ]:
# splitting the dataset into train and test split : 
training_size = int(len(data1)*0.65)

In [ ]:
test_size = len(data1) - training_size

In [ ]:
train_data,test_data = data1[0:training_size,:],data1[training_size:len(data1),:1]

In [ ]:
len(train_data)

In [ ]:
len(test_data)

In [ ]:
# converting the array values into a dataset matrix : 
def create_dataset(dataset, time_step = 1):
    dataX , dataY = [], []
    for i in range(len(dataset) - time_step - 1 ):
        a = dataset[i:(i + time_step), 0]  
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 100

In [ ]:
X_train, y_train = create_dataset(train_data, time_step)

X_test, y_test = create_dataset(test_data, time_step)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
# reshape the input to be in [samples, time steps, features] which is required for LSTM
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)

In [ ]:
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

# CREATING THE STACKED LSTM MODEL

In [ ]:
model=Sequential()

In [ ]:
model.add(LSTM(50,return_sequences = True,input_shape = (100,1)))
model.add(LSTM(50,return_sequences = True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer = 'adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 100,batch_size = 64,verbose = 1)

In [ ]:
# prediction and performance metrics:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

In [ ]:
# Back to original form : 
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

In [ ]:
# RMSE performance metrics (Training dataset): 
math.sqrt(mean_squared_error(y_train,train_predict))

In [ ]:
# RMSE performance metrics (Test dataset): 
math.sqrt(mean_squared_error(y_test,test_predict))

In [ ]:
# shifting train predictions for plotting
look_back=100
trainPredictPlot = np.empty_like(data1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
# shifting test predictions for plotting
testPredictPlot = np.empty_like(data1)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(data1)-1, :] = test_predict

In [ ]:
# ploting baseline and predictions
plt.plot(scaler.inverse_transform(data1) , color = "yellow")
plt.plot(trainPredictPlot , color = "blue") # train predict data
plt.plot(testPredictPlot  , color = "red") # test predict data
plt.show()

In [ ]:
# previous 100 days:
x_input = test_data[341:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input = list(x_input)

In [ ]:
temp_input = temp_input[0].tolist()

In [ ]:
# demonstrate prediction for next 30 days
lst_output=[]
n_steps=100
i=0
while(i<30):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input = np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        # print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        # print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

print(lst_output)

In [ ]:
day_new=np.arange(1,101)

In [ ]:
day_pred=np.arange(101,131)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(data1[1158:]) , color="green")
plt.plot(day_pred,scaler.inverse_transform(lst_output) , color = "orange")

In [ ]:
data3 = data1.tolist()
data3.extend(lst_output)
plt.plot(data3[1200:] , color = "green")